In [3]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
import os
from scipy.optimize import minimize

# Matplotlib Einstellungen gemäß den LaTeX-Caption-Formatierungen
plt.rcParams.update({
    'text.usetex': True,              # Enable LaTeX for text rendering
    'text.latex.preamble': r'\usepackage{amsmath}',
    'font.family': 'serif',           # Use a serif font family
    'font.serif': 'Palatino',         # Set Palatino as the serif font
    'font.size': 20,                   # Font size for general text
    'axes.titlesize': 20,              # Font size for axis titles
    'axes.labelsize': 20,              # Font size for axis labels
    'xtick.labelsize': 20,             # Font size for x-axis tick labels
    'ytick.labelsize': 20,             # Font size for y-axis tick labels
    'legend.fontsize': 20,             # Font size for legends
    'figure.figsize': [8, 6],          # Size of the plot (width x height)
    'figure.autolayout': True,         # Automatic layout adjustment
    'savefig.format': 'svg',           # Default format for saving figures
    'figure.facecolor': 'none',        # Make the figure face color transparent
    'axes.facecolor': 'none',          # Make the axes face color transparent
    'savefig.transparent': True        # Save figures with transparent background
})

output_dir = r"C:\Users\leopold\OneDrive - UT Cloud\Uni\Semester_8\BA_mit_Git\BA_Latex\Figures_From_Python"
os.makedirs(output_dir, exist_ok=True)

########################################                 Define constants                   #############################################
fixed_lam = 1
fixed_gamma = 1
fixed_dist_ext = 0.1 * fixed_lam
k_a = 2 * np.pi / fixed_lam

In [4]:
######################################################### Functions #########################################################
def dipole_vector(phi):
    return np.array([np.cos(phi), np.sin(phi), 0])

def z_rotation(angle):
    return np.array([[np.cos(angle), -np.sin(angle), 0],
                     [np.sin(angle),  np.cos(angle), 0],
                     [0,              0,             1]])

def chain_positions(distance, N_atoms):
    Pos = np.zeros((N_atoms, 3))
    for i in range(N_atoms):
        Pos[i, 0] = i * distance
    return Pos

def topo_positions_only_1_inner(distance_in, N_atoms):
    if distance_in >= 2 * fixed_dist_ext:
        distance_in = 2 * fixed_dist_ext - 1e-5
    Pos = np.zeros((N_atoms, 3))
    N_chain = N_atoms // 3
    height = np.sqrt(fixed_dist_ext**2 - (distance_in / 2)**2)
    r_in = (2 * fixed_dist_ext * distance_in - distance_in**2)/(4*height)
    MB = np.sqrt(r_in**2 + (distance_in / 2)**2)
    angle = np.arccos(((r_in) / MB))
    A = np.array([height - r_in, 0, 0])
    B = np.array([r_in, distance_in/2, 0])
    C = np.array([r_in, -distance_in/2, 0])
    Chain = chain_positions(fixed_dist_ext, N_chain)
    Pos[:N_chain] = np.dot(Chain + A, z_rotation(np.pi).T)
    Pos = Pos[Pos[:, 0].argsort()]
    Pos[N_chain:2*N_chain] = np.dot(Chain, z_rotation(angle).T) + B
    Pos[2*N_chain:] = np.dot(Chain, z_rotation(-angle).T) + C
    return Pos

def Green_tensor(r_a, r_b):
    r_ab = r_b - r_a
    abs_r_ab = np.linalg.norm(r_ab)
    kappa = k_a * abs_r_ab
    return (np.exp(1j * kappa) / (4 * np.pi * kappa ** 2 * abs_r_ab)
                    * ((kappa ** 2 + 1j * kappa - 1) * np.eye(3)
                       + (- kappa ** 2 - 3 * 1j * kappa + 3)
                       * np.outer(r_ab, r_ab) / (abs_r_ab ** 2)))

def Gamma_matrix(distance_in, dipoles, N_atoms):
    positions = topo_positions_only_1_inner(distance_in, N_atoms)
    G_matrix = np.zeros((N_atoms, N_atoms), dtype=complex)
    for a in range(N_atoms):
        for b in range(N_atoms):
            r_a, r_b = positions[a], positions[b]
            G_matrix[a, b] = fixed_gamma
            if np.linalg.norm(r_b - r_a) > 1e-5:
                d_a, d_b = dipoles[a], dipoles[b]
                k_a = 2 * np.pi / fixed_lam
                G_matrix[a, b] = np.imag((6 * np.pi * fixed_gamma / k_a * np.matmul(np.conj(d_a), np.matmul(Green_tensor(r_a, r_b), d_b.T))))
    return G_matrix

def V_matrix(distance_in, dipoles, N_atoms):
    positions = topo_positions_only_1_inner(distance_in, N_atoms)
    V_matrix = np.zeros((N_atoms, N_atoms), dtype=complex)
    for a in range(N_atoms):
        for b in range(N_atoms):
            r_a, r_b = positions[a], positions[b]
            V_matrix[a, b] = 0
            if np.linalg.norm(r_b - r_a) > 1e-5:
                d_a, d_b = dipoles[a], dipoles[b]
                k_a = 2 * np.pi / fixed_lam
                V_matrix[a, b] = - np.real((3 * np.pi * fixed_gamma / k_a * np.matmul(np.conj(d_a), np.matmul(Green_tensor(r_a, r_b), d_b.T))))
    return V_matrix

def H_eff(distance_in, angle):
    dipoles = [dipole_vector(angle) for _ in range(N_atoms)]
    G = Gamma_matrix(distance_in, dipoles, N_atoms)
    V = V_matrix(distance_in, dipoles, N_atoms)
    # Generate connections based on the updated rules
    return Qobj(V) - 1j / 2 * Qobj(G)
'''    connections = {}
    for i in range(N_atoms):
        connections[i] = [i]
        if i - 1 >= 0:
            connections[i].append(i - 1)
        if i + 1 < N_atoms:
            connections[i].append(i + 1)        
        if i == N_atoms // 3 - 1:
            connections[i].append(2 * N_atoms // 3)
        elif i == N_atoms // 3:
            connections[i].append(2 * N_atoms // 3)
        elif i == 2 * N_atoms // 3:
            if i - 1 in connections[i]:
                connections[i].remove(i - 1)
            connections[i].extend([N_atoms // 3, N_atoms // 3 - 1])
        connections[i] = sorted(list(set(connections[i])))

    for i in range(N_atoms):
        for j in range(N_atoms):
            if j not in connections[i]:
                x = 0
#                G[i, j] = 0
#                V[i, j] = 0'''

def survival_probabilities(times, angle, distance_in, Psi_0, N_atoms):
    H = H_eff(distance_in, angle)
    coeffs_sq_mods  = np.zeros((len(times), N_atoms))
    P_surs  = np.zeros(len(times))
    for t_idx, t in enumerate(times):
        U = (-1j * H * t).expm()
        Psi_t = (U * Psi_0).full().flatten()
        Probs = np.abs(Psi_t)**2
        coeffs_sq_mods[t_idx, :] = Probs
        P_surs[t_idx] = Probs.sum()
    return coeffs_sq_mods, P_surs

# Hamilton function where all dipoles have the same angle
def Get_V_triangle(distance_in, angle, N_atoms=3):
    dipoles = [dipole_vector(angle) for _ in range(N_atoms)]
    idx1 = N_atoms//3 - 1
    idx2 = N_atoms//3
    idx3 = 2 * N_atoms//3
    V = V_matrix(distance_in, dipoles, N_atoms)
    return V[idx1, idx2], V[idx2, idx3], V[idx1, idx3]

# Target function to minimize V13/V12 and V23/V12 for a given distance and common angle
def target_function(angle, distance_in):
    V12, V23, V13 = Get_V_triangle(distance_in, angle[0])
    # Avoid division by zero
    if np.abs(V12) > 1e-5:
        # Compute the ratios
        ratio_V13_V12 = np.abs(V13) / np.abs(V12)
        ratio_V23_V12 = np.abs(V23) / np.abs(V12)
        
        # We want to minimize both ratios, so we return their sum
        return ratio_V13_V12 + ratio_V23_V12
    else:
        # If V12 is too small, return a large value (penalty)
        return np.inf

# Function to optimize over angles for a fixed distance
def find_optimal_angle_for_distance(distance_in):
    # Initial guess for the angle as a scalar
    initial_guess = np.pi / 4  # Arbitrary starting guess
    bounds = [(0, np.pi)]  # Bounds for the angle
    
    # Perform the optimization using scipy's minimize function
    result = minimize(target_function, initial_guess, args=(distance_in,), bounds=bounds)
    return result.x[0], result.fun  # Return optimized angle and the minimized value

In [5]:
#
# Find the optimal values of d_in, theta that minimize V13, V23, and V12=/=0
#
N_atoms = 3
# Scanning over distances to find the best angle
num_distance_points = 100  # Number of distance points
distances = np.linspace(fixed_dist_ext / 10, 2 * fixed_dist_ext, num_distance_points) 

# Arrays to store results
optimal_angles = []
minimized_values = []
V12_values = []
V13_values = []
V23_values = []
ratio_V13_V12 = []
ratio_V23_V12 = []

for distance_in in distances:
    opt_angle, min_value = find_optimal_angle_for_distance(distance_in)
    optimal_angles.append(opt_angle)
    minimized_values.append(min_value)
    
    V12, V23, V31 = Get_V_triangle(distance_in, opt_angle)
    
    # Store the absolute values of the Hamiltonian elements
    V12_values.append(np.abs(V12))
    V13_values.append(np.abs(V31))
    V23_values.append(np.abs(V23))
    
    # Compute the ratios if V12 is not too small (to avoid division by zero)
    if np.abs(V12) > 1e-5:
        ratio_V13_V12.append(np.abs(V31) / np.abs(V12))
        ratio_V23_V12.append(np.abs(V23) / np.abs(V12))
    else:
        ratio_V13_V12.append(np.nan)  # Append NaN for undefined ratios
        ratio_V23_V12.append(np.nan)


# Find the index of the minimum value in minimized_values
min_index = np.argmin(minimized_values[:])

# Get the corresponding distance and angle
minimized_value = minimized_values[min_index]
optimal_distance_in = distances[min_index]
optimal_angle = optimal_angles[min_index]
opt_V12_value = V12_values[min_index]
opt_V13_value = V13_values[min_index]
opt_V23_value = V23_values[min_index]

# Define the exclusion range (15 indices before and after the min_index)
exclusion_range = list(range(max(0, min_index - 15), min(len(minimized_values), min_index + 16)))

# Create a copy of minimized_values and set the values in the exclusion range to infinity
minimized_values_copy = np.array(minimized_values[:])
minimized_values_copy[exclusion_range] = np.inf

# Find the index of the second smallest value outside the exclusion range
second_min_index = np.argmin(minimized_values_copy)

# Get the corresponding distance and angle for the second smallest value
second_minimized_value = minimized_values[second_min_index]
second_optimal_distance_in = distances[second_min_index]
second_optimal_angle = optimal_angles[second_min_index]
second_opt_V12_value = V12_values[second_min_index]
second_opt_V13_value = V13_values[second_min_index]
second_opt_V23_value = V23_values[second_min_index]


# Print the results
print(f"fixed d_(ext): {fixed_dist_ext}")
print(f"minimized value: {minimized_value}")
print(f"Corresponding distance: {optimal_distance_in}")
print(f"Corresponding optimal angle: {optimal_angle} radians")

print(f"V12: {opt_V12_value}")
print(f"V13: {opt_V13_value}")
print(f"V23: {opt_V23_value}")


# Print the results for the second minimum
print(f"Second minimized value: {second_minimized_value}")
print(f"Corresponding second distance: {second_optimal_distance_in}")
print(f"Corresponding second optimal angle: {second_optimal_angle} radians")
print(f"V12 (second): {second_opt_V12_value}")
print(f"V13 (second): {second_opt_V13_value}")
print(f"V23 (second): {second_opt_V23_value}")


# Plotting the optimal angles and minimized values as a function of distance
fig, axs = plt.subplots(4, 1, figsize=(8, 10), sharex=True, constrained_layout=True)

# Plot for optimal angles
axs[0].plot(distances / fixed_dist_ext, optimal_angles, label='Optimal Angle', color='b')
axs[0].axvline(optimal_distance_in / fixed_dist_ext, color='r', linestyle='--')
axs[0].set_ylabel('Optimal Angle (radians)')
axs[0].grid(True)
axs[0].legend()

# Plot for minimized values
axs[1].plot(distances / fixed_dist_ext, minimized_values, label='Minimized Ratios', color='r')
axs[1].axvline(optimal_distance_in / fixed_dist_ext, color='r', linestyle='--')
axs[1].set_ylabel('Minimized Value (Ratios)')
axs[1].grid(True)
axs[1].legend()

# Plotting the Hamiltonian values
axs[2].plot(distances / fixed_dist_ext, V12_values, label='(V12)', color='b')
axs[2].plot(distances / fixed_dist_ext, V13_values, label='(V31)', color='r', linestyle='--')
axs[2].plot(distances / fixed_dist_ext, V23_values, label='(V23)', color='g', linestyle='--')
axs[2].axvline(optimal_distance_in / fixed_dist_ext, color='r', linestyle='--')
axs[2].set_ylabel('Hamiltonian Values')
axs[2].grid(True)
axs[2].legend()

# Plotting the ratios of Hamiltonian values
axs[3].plot(distances / fixed_dist_ext, ratio_V13_V12, label='(V13 / V12)', color='r')
axs[3].plot(distances / fixed_dist_ext, ratio_V23_V12, label='(V23 / V12)', color='g')
axs[3].axvline(optimal_distance_in / fixed_dist_ext, color='r', linestyle='--')
axs[3].set_xlabel('d / d_ext')
#axs[3].set_ylabel('Ratios of Hamiltonian Values')
#axs[3].grid(True)
axs[3].legend()

# Show the combined figure
figure_name = f"TESTTt.svg"
#plt.savefig(os.path.join(output_dir, figure_name))
plt.show()


# Plot atoms with their dipole moments in the seperate plot
positions = topo_positions_only_1_inner(optimal_distance_in, N_atoms)
dipoles = [dipole_vector(optimal_angle) for _ in range(N_atoms)]
plt.figure()
plt.scatter(positions[:, 0], positions[:, 1], color='blue', s=50, label='Atoms')
for i in range(N_atoms):
    plt.arrow(positions[i, 0], positions[i, 1], 0.1*dipoles[i][0], 0.1*dipoles[i][1])
plt.title('Atom Positions and Dipole Moments')
plt.xlabel('x / $\lambda$')
plt.ylabel('y / $\lambda$')
figure_name = f"ASYMM_TOPO_Atom_Dipole_Pos_N={N_atoms}_angle={optimal_angle:.2f}_distance={optimal_distance_in:.2f}_wo_description.svg"
#plt.savefig(os.path.join(output_dir, figure_name))
plt.axis('equal')
plt.show()

In [5]:
# Create figure and axis
fig, ax = plt.subplots(figsize=(4, 3))

ratio_V13_V12 = np.array(ratio_V13_V12)
ratio_V23_V12 = np.array(ratio_V23_V12)
# Lower plot: Ratios of Hamiltonian values (axs[1])
#ax.scatter(distances, ratio_V13_V12, label='(V13 / V12)', color='#1f77b4', s=30, marker='o', alpha=0.8, edgecolor='#0177C9', linewidth=1.5)
#ax.scatter(distances, ratio_V23_V12, label='(V23 / V12)', color='#ff7f0e', s=30, marker='o', alpha=0.8, edgecolor='#FF7F02', linewidth=1.5)
ax.scatter(distances, ratio_V13_V12 + ratio_V23_V12, color='#ff7f0e', s=50, marker='o', alpha=0.8, edgecolor='#0177C9', linewidth=1.5)
ax.axvline(optimal_distance_in , color='r', linestyle='--')
ax.axvline(second_optimal_distance_in, color='g', linestyle='--')

# Set limits and ticks for x and y axes on the second plot
x_min, x_max = distances.min(), distances.max()
y_max = max(np.max(ratio_V13_V12), np.max(ratio_V23_V12))
ax.set_yscale('log')
ax.set_xlim(0.01, x_max)
#ax.set_ylim(0, 0.1)

# Set custom ticks
ax.set_xticks([0.01, 0.1, x_max])
ax.set_yticks([0.001, 0.1, y_max])

# Set labels and legend
ax.set_xlabel(r'$d_{\text{in}} / \lambda$')
ax.set_ylabel(r'$\frac{V_{13}}{V_{12}}+\frac{V_{23}}{V_{12}}$')

figure_name = f"Ratios_ASYMM.svg"
plt.savefig(os.path.join(output_dir, figure_name))

plt.show()

In [11]:
# Function to compute angle between dipole and connection vector
def compute_angle_between_vectors(pos1, pos2, dipole_norm):
    connection_vector = pos2 - pos1
    connection_vector_norm = connection_vector / np.linalg.norm(connection_vector)
    cos_theta = np.dot(dipole_norm, connection_vector_norm)
    angle_rad = np.arccos(cos_theta)
    angle_deg = np.degrees(angle_rad)
    return angle_deg, angle_rad

# Compute angles for pairs of atoms: 1-3, 1-2, and 2-3
dipole = dipole_vector(optimal_angle)
dipole_norm = dipole / np.linalg.norm(dipole)
angle_13_deg, angle_13_rad = compute_angle_between_vectors(positions[0], positions[2], dipole_norm)
angle_12_deg, angle_12_rad = compute_angle_between_vectors(positions[0], positions[1], dipole_norm)
angle_23_deg, angle_23_rad = compute_angle_between_vectors(positions[1], positions[2], dipole_norm)

# Display the results
print(f"Angle between dipoles and connection vector:\n1,3:{angle_13_deg:.4f} \n1,2:{angle_12_deg:.4f}\n2,3:{angle_23_deg:.4f}")

In [24]:
# Define the threshold
threshold = 0.01

# Calculate the number of ratios below the threshold for each case
count_below_threshold_V13_V12 = sum(1 for ratio in ratio_V13_V12 if not np.isnan(ratio) and ratio < threshold)
count_below_threshold_V23_V12 = sum(1 for ratio in ratio_V23_V12 if not np.isnan(ratio) and ratio < threshold)

# Calculate the total number of valid ratios (excluding NaNs)
total_valid_V13_V12 = sum(1 for ratio in ratio_V13_V12 if not np.isnan(ratio))
total_valid_V23_V12 = sum(1 for ratio in ratio_V23_V12 if not np.isnan(ratio))

# Calculate the percentage of ratios below the threshold
percent_below_threshold_V13_V12 = (count_below_threshold_V13_V12 / total_valid_V13_V12) * 100 if total_valid_V13_V12 > 0 else 0
percent_below_threshold_V23_V12 = (count_below_threshold_V23_V12 / total_valid_V23_V12) * 100 if total_valid_V23_V12 > 0 else 0

# Print the results
print(f"Percentage of V13/V12 ratios below {threshold}: {percent_below_threshold_V13_V12:.2f}%")
print(f"Percentage of V23/V12 ratios below {threshold}: {percent_below_threshold_V23_V12:.2f}%")

In [6]:
# Find the index where distance is closest to 0.1
target_distance = 0.1
closest_index = np.abs(distances - target_distance).argmin()

# Get the corresponding optimal angle
closest_distance = distances[closest_index]
optimal_angle_at_closest_distance = optimal_angles[closest_index]

closest_distance, optimal_angle_at_closest_distance